<a href="https://colab.research.google.com/github/emamarela/ActiveLearning/blob/main/DistilRoberta_MultiEurlex_EN_BreakingTies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/webis-de/small-text

Cloning into 'small-text'...
remote: Enumerating objects: 2050, done.
remote: Counting objects: 100% (2050/2050), done.
remote: Compressing objects: 100% (1352/1352), done.
remote: Total 2050 (delta 1334), reused 1400 (delta 684), pack-reused 0
Receiving objects: 100% (2050/2050), 403.12 KiB | 13.44 MiB/s, done.
Resolving deltas: 100% (1334/1334), done.


In [ ]:
import sys
sys.path.insert(0, '/content/small-text')
sys.path.insert(0, '/content/small-text/examples')

In [ ]:
!pip install transformers
!pip install datasets
!pip install numpy==1.20.0

     |████████████████████████████████| 3.5 MB 9.6 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 6.8 MB 66.0 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 9.0 MB/s 
     |████████████████████████████████| 243 kB 85.4 MB/s 
     |████████████████████████████████| 133 kB 91.6 MB/s 
     |████████████████████████████████| 1.1 MB 92.7 MB/s 
     |████████████████████████████████| 271 kB 95.1 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
     |████████████████████████████████| 144 kB 73.0 MB/s 
     |████████████████████████████████| 15.3 MB 7.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21

# New Section

In [ ]:
import logging
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

from small_text.active_learner import PoolBasedActiveLearner

from small_text.initialization import random_initialization_stratified
from small_text.integrations.transformers import TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import PoolExhaustedException, EmptyPoolException
from small_text.query_strategies import RandomSampling, EmbeddingKMeans, LeastConfidence,BreakingTies

from examplecode.data.example_data_multilabel import get_train_test
from examplecode.data.example_data_transformers import preprocess_data
from examplecode.shared import evaluate_multi_label

from sklearn.metrics import f1_score

#TRANSFORMER_MODEL = TransformerModelArguments('distilroberta-base') # for electra: 'google/electra-base-discriminator'


In [ ]:
TRANSFORMER_MODEL = TransformerModelArguments('distilroberta-base')

num_classes = 21
clf_factory = TransformerBasedClassificationFactory(TRANSFORMER_MODEL,
                                                    num_classes,
                                                    kwargs=dict({
                                                        'device': 'cuda',
                                                        'multi_label': True
                                                    }))
query_strategy = BreakingTies()

tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL.model, cache_dir='.cache/')

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:

def perform_active_learning(active_learner, train, labeled_indices, test):
    # Perform 10 iterations of active learning...
    for i in range(10):
        # ...where each iteration consists of labelling 20 samples
        q_indices = active_learner.query(num_samples=100)

        # Simulate user interaction here. Replace this for real-world usage.
        y = train.y[q_indices]

        # Return the labels for the current query to the active learner.
        active_learner.update(y)

        labeled_indices = np.concatenate([q_indices, labeled_indices])

        print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
        evaluate_multi_label(active_learner, train[labeled_indices], test)
        #results.append(evaluate_multi_label(active_learner, train[labeled_indices], test))


In [ ]:
def initialize_active_learner(active_learner, y_train):

    x_indices_initial = random_initialization_stratified(y_train, n_samples=800)
    y_initial = y_train[x_indices_initial]

    active_learner.initialize_data(x_indices_initial, y_initial)

    return x_indices_initial


In [ ]:
eurlex_en = load_dataset('multi_eurlex', 'en', cache_dir='/content/drive/MyDrive')

Downloading:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset multi_eurlex downloaded and prepared to /content/drive/MyDrive/multi_eurlex/en/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def main():
    # Active learning parameters
    num_classes = 21
    clf_factory = TransformerBasedClassificationFactory(TRANSFORMER_MODEL,
                                                        num_classes,
                                                        kwargs=dict({
                                                            'device': 'cuda',
                                                            'multi_label': True
                                                        }))
    query_strategy = BreakingTies()

    # Prepare some data
    train = eurlex_en['train']
    test = eurlex_en['validation']
    
    tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL.model, cache_dir='.cache/')
    x_train = preprocess_data(tokenizer, train['text'], train['labels'], multi_label=True)
    x_test = preprocess_data(tokenizer, test['text'], test['labels'], multi_label=True)
    # Active learner
    active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, x_train)
    
    
    labeled_indices = initialize_active_learner(active_learner, x_train.y)

    try:
        perform_active_learning(active_learner, x_train, labeled_indices, x_test)
    except PoolExhaustedException:
        print('Error! Not enough samples left to handle the query.')
    except EmptyPoolException:
        print('Error! No more samples left. (Unlabeled pool is empty)')


In [ ]:
if __name__ == '__main__':
    logging.getLogger('small_text').setLevel(logging.INFO)
    #logging.getLogger('transformers.modeling_utils').setLevel(logging.ERROR)

    main()


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

...

Iteration #7 (1600 samples)
Train accuracy: 0.86
Test accuracy: 0.69
---


INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 1 | 00:00:46
	Train Set Size: 1530
	Loss: 0.0301(train)	|	Acc: 87.2%(train)
	Loss: 0.0239(valid)	|	Acc: 90.5%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 2 | 00:00:45
	Train Set Size: 1530
	Loss: 0.0201(train)	|	Acc: 91.9%(train)
	Loss: 0.0199(valid)	|	Acc: 92.5%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 3 | 00:00:45
	Train Set Size: 1530
	Loss: 0.0170(train)	|	Acc: 93.2%(train)
	Loss: 0.0179(valid)	|	Acc: 93.3%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 4 | 00:00:45
	Train Set Size: 1530
	Loss: 0.0148(train)	|	Acc: 94.2%(train)
	Loss: 0.0166(valid)	|	Acc: 94.1%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 5 | 00:00:45
	Train Set Size: 1530
	Loss: 0.0134(train)	|	Acc: 94.8%(train)
	Loss: 0.0160(valid)	|	Acc: 94.3%(valid)
INFO:small_text.integrations.transf

Iteration #8 (1700 samples)
Train accuracy: 0.86
Test accuracy: 0.69
---


INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 1 | 00:00:48
	Train Set Size: 1620
	Loss: 0.0282(train)	|	Acc: 88.0%(train)
	Loss: 0.0232(valid)	|	Acc: 88.9%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 2 | 00:00:48
	Train Set Size: 1620
	Loss: 0.0197(train)	|	Acc: 91.8%(train)
	Loss: 0.0194(valid)	|	Acc: 91.7%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 3 | 00:00:48
	Train Set Size: 1620
	Loss: 0.0165(train)	|	Acc: 93.2%(train)
	Loss: 0.0161(valid)	|	Acc: 93.3%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 4 | 00:00:48
	Train Set Size: 1620
	Loss: 0.0145(train)	|	Acc: 94.3%(train)
	Loss: 0.0148(valid)	|	Acc: 93.4%(valid)
INFO:small_text.integrations.transformers.classifiers.classification:Epoch: 5 | 00:00:48
	Train Set Size: 1620
	Loss: 0.0130(train)	|	Acc: 94.9%(train)
	Loss: 0.0138(valid)	|	Acc: 94.0%(valid)
INFO:small_text.integrations.transf

Iteration #9 (1800 samples)
Train accuracy: 0.87
Test accuracy: 0.70
---
